---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [1]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')
print(spam_data.head())
spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)
spam_data['text'][4]

                                                text target
0  Go until jurong point, crazy.. Available only ...    ham
1                      Ok lar... Joking wif u oni...    ham
2  Free entry in 2 a wkly comp to win FA Cup fina...   spam
3  U dun say so early hor... U c already then say...    ham
4  Nah I don't think he goes to usf, he lives aro...    ham


"Nah I don't think he goes to usf, he lives around here though"

In [2]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [168]:
def answer_one():
    #n_spam = np.sum(spam_data['target'])
    #n_tot = spam_data['target'].shape[0]
    return spam_data['target'].mean()*100 #Your answer here
#answer_one()

In [169]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [180]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    vect = CountVectorizer().fit(X_train)
    names = vect.get_feature_names()
    #print(names)
    length = [len(x) for x in names]
    max_len = max(length)
    result = [x for x in names if len(x) == max_len]
    return result[0]#Your answer here

In [181]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [7]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    vect = CountVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    
    MNB = MultinomialNB(alpha=0.1)
    MNB.fit(X_train_vect, y_train)
    
    X_test_vect = vect.transform(X_test)
    y_pred = MNB.predict_proba(X_test_vect)
    #print(y_pred)
    #print(MNB.classes_)
    
    y_pred = [x[1] for x in y_pred]
    auc = roc_auc_score(y_test, y_pred)
    return auc #Your answer here

In [8]:
answer_three()

0.99154542213469599

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [182]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    vect = TfidfVectorizer().fit(X_train)
    X_train_vect = vect.transform(X_train)
    
    feature_names = np.array(vect.get_feature_names())
    max_tfidf_index = X_train_vect.max(0).toarray()[0].argsort()
    min_tfidf_index = X_train_vect.min(0).toarray()[0].argsort()
    largest = feature_names[max_tfidf_index[-20:]] 
    smallest = feature_names[min_tfidf_index[:20]]
    
    largest_s = pd.Series(data=X_train_vect.max(0).toarray()[0][max_tfidf_index[-20:]], index = largest)
    smallest_s = pd.Series(data=X_train_vect.min(0).toarray()[0][min_tfidf_index[:20]], index = smallest)
    
    #sort two series
    df_large = largest_s.reset_index(drop=False)
    df_large = df_large.sort_values(by=[0,'index'], ascending=[False, True])
    s_large = df_large.set_index('index')[0]
    df_small = smallest_s.reset_index(drop=False)
    df_small = df_small.sort_values(by=[0, 'index'], ascending=[True, True])
    s_small = df_small.set_index('index')[0]
    result = (s_small, s_large)
    return result#Your answer here

In [183]:
answer_four()

(index
 00                 0.0
 petrol             0.0
 petticoatdreams    0.0
 pg                 0.0
 ph                 0.0
 pharmacy           0.0
 phb1               0.0
 phews              0.0
 phil               0.0
 philosophical      0.0
 philosophy         0.0
 phne               0.0
 phoenix            0.0
 phone              0.0
 phonebook          0.0
 phoned             0.0
 phones             0.0
 phony              0.0
 photo              0.0
 photos             0.0
 Name: 0, dtype: float64, index
 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 Name: 0, dtype: floa

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [11]:
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
def answer_five():
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_vect = vect.transform(X_train)
    model = MultinomialNB(alpha=0.1).fit(X_train_vect, y_train)
    X_test_vect = vect.transform(X_test)
    y_pred = model.predict(X_test_vect)
    auc = roc_auc_score(y_test, y_pred)
    return auc#Your answer here

In [12]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [13]:
def answer_six():
    spam = spam_data[spam_data['target']==1]['text'].tolist()
    spam_len = np.mean([len(x) for x in spam])
    non_spam = spam_data[spam_data['target']==0]['text'].tolist()
    non_spam_len = np.mean([len(x) for x in non_spam])
    return non_spam_len, spam_len#Your answer here

In [14]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [15]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [16]:
from sklearn.svm import SVC

def answer_seven():
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_vect = vect.transform(X_train)
    x_new_ft = [len(x) for x in X_train]
    X_train_vect2 = add_feature(X_train_vect, x_new_ft)
    #print(X_train_vect2)
    model = SVC(C=10000).fit(X_train_vect2, y_train)
    
    X_test_vect = vect.transform(X_test)
    x_test_ft = [len(x) for x in X_test]
    X_test_vect2 = add_feature(X_test_vect, x_test_ft)
    y_pred = model.predict(X_test_vect2)
    auc = roc_auc_score(y_test, y_pred)
    return auc#Your answer here

In [17]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [18]:
def answer_eight():
    spam = spam_data[spam_data['target']==1]['text'].tolist()
    spam_digit = []
    for text in spam:
        spam_digit.append([x for x in text if str.isdigit(x)])
    spam_len = np.mean([len(x) for x in spam_digit])
    #print(spam_digit)
    
    non_spam = spam_data[spam_data['target']==0]['text'].tolist()
    non_spam_digit = []
    for text in non_spam:
        non_spam_digit.append([x for x in text if str.isdigit(x)])
    non_spam_len = np.mean([len(x) for x in non_spam_digit])
    #print(non_spam_digit)
    
    return (non_spam_len, spam_len)#Your answer here

In [19]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [20]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    X_train_vect = vect.transform(X_train)
    x_new_ft1 = [len(x) for x in X_train]
    #print(x_new_ft1)
    digit = []
    for text in X_train:
        digit.append([x for x in text if str.isdigit(x)]) 
    x_new_ft2 = [len(x) for x in digit]
    #print(x_new_ft2)
    X_train_vect2 = add_feature(X_train_vect, [x_new_ft1, x_new_ft2])
    #print(X_train_vect2[1,3384])
    
    model = LogisticRegression(C=100).fit(X_train_vect2, y_train)
    
    X_test_vect = vect.transform(X_test)
    x_new_ft1_test = [len(x) for x in X_test]
    digit = []
    for text in X_test:
        digit.append([x for x in text if str.isdigit(x)]) 
    x_new_ft2_test = [len(x) for x in digit]
    X_test_vect2 = add_feature(X_test_vect, [x_new_ft1_test, x_new_ft2_test])
    y_pred = model.predict(X_test_vect2)
    
    auc = roc_auc_score(y_test, y_pred)
    
    return auc#Your answer here

In [21]:
answer_nine()

0.96533283533945646

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [192]:
#import re
def answer_ten():
    import re
    spam = spam_data[spam_data['target']==1]['text'].tolist()
    spam_n_word = []
    for text in spam:
        spam_n_word.append(re.sub('\w', '', text))
    spam_len = np.mean([len(x) for x in spam_n_word])
    #print(spam_n_word[0])
    #print(spam[0])
    
    ham = spam_data[spam_data['target']==0]['text'].tolist()
    ham_n_word = []
    for text in ham:
        ham_n_word.append(re.sub('\w', '', text))
    ham_len = np.mean([len(x) for x in ham_n_word])
    
    return ham_len, spam_len#Your answer here

In [193]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [166]:
def answer_eleven():
    import re
    vect = CountVectorizer(min_df=5, ngram_range=(2,5), analyzer='char_wb').fit(X_train)
    X_train_vect = vect.transform(X_train)
    #print(len(vect.get_feature_names()))
    
    # create three new features
    length_of_doc = [len(x) for x in X_train]
    digit = []
    n_digit = []
    for text in X_train:
        digit.append([x for x in text if str.isdigit(x)]) 
        n_digit.append(re.sub('[A-Za-z0-9_]', '', text))
    digit_count = [len(x) for x in digit]
    non_word_char_count = [len(x) for x in n_digit]
    
    X_train_vect2 = add_feature(X_train_vect, [length_of_doc, digit_count, non_word_char_count])
    #print(X_train_vect)
    #print('-------------------------')
    #print(X_train_vect2)
    
    # fit model and predict
    model = LogisticRegression(C=100).fit(X_train_vect2, y_train)
    X_test_vect = vect.transform(X_test)
    length_of_doc_test = [len(x) for x in X_test]
    digit_test = []
    n_digit_test = []
    for text in X_test:
        digit_test.append([x for x in text if str.isdigit(x)]) 
        n_digit_test.append(re.sub('[A-Za-z0-9_]', '', text))
    digit_count_test = [len(x) for x in digit_test]
    non_word_char_count_test = [len(x) for x in n_digit_test]
    
    X_test_vect2 = add_feature(X_test_vect, [length_of_doc_test, digit_count_test, non_word_char_count_test])
    y_pred = model.predict(X_test_vect2)
    auc = roc_auc_score(y_test, y_pred)
    
    # find smallest and largest coeff
    coef_order = model.coef_.argsort()[0]
    feature_name = np.array(vect.get_feature_names() + ['length_of_doc', 'digit_count', 'non_word_char_count'])
    
    smallest_coef = feature_name[coef_order[:10]].tolist()
    largest_coef = feature_name[coef_order[-1:-11:-1]].tolist()
    return auc, smallest_coef, largest_coef #Your answer here

In [167]:
answer_eleven()

(0.97885931107074342,
 ['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'],
 ['digit_count', 'ne', 'co', 'ia', 'xt', ' ch', ' x', 'mob', 'ww', 'ar'])